In [4]:
import sqlite3
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import tkcalendar as tl
from tkcalendar import DateEntry
import matplotlib.pyplot as plt
import pandas as pd


In [5]:
def init_db():
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS expenses (
                        id INTEGER PRIMARY KEY,
                        date TEXT,
                        category TEXT,
                        amount REAL,
                        description TEXT)''')
    conn.commit()
    conn.close()


In [6]:
def add_expense(date, category, amount, description):
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO expenses (date, category, amount, description) VALUES (?, ?, ?, ?)",
                   (date, category, float(amount), description))
    conn.commit()
    conn.close()



In [7]:
def get_expenses():
    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM expenses")
    rows = cursor.fetchall()
    conn.close()
    return rows


In [8]:
# GUI Setup
root = tk.Tk()
root.title("Personal Expense Tracker")
root.geometry("800x500")



''

In [9]:

frame = ttk.Frame(root, padding=10)
frame.pack(fill=tk.BOTH, expand=True)

# Input Fields
ttk.Label(frame, text="Date:").grid(row=0, column=0)
date_entry = DateEntry(frame)
date_entry.grid(row=0, column=1)


In [10]:
category_var = tk.StringVar()
ttk.Label(frame, text="Category:").grid(row=0, column=2)
category_menu = ttk.Combobox(frame, textvariable=category_var, values=["Food", "Transport", "Bills", "Shopping", "Others"])
category_menu.grid(row=0, column=3)

amount_entry = ttk.Entry(frame)
ttk.Label(frame, text="Amount:").grid(row=1, column=0)
amount_entry.grid(row=1, column=1)

description_entry = ttk.Entry(frame)
ttk.Label(frame, text="Description:").grid(row=1, column=2)
description_entry.grid(row=1, column=3)


In [11]:
# Functions
def add_expense_gui():
    date = date_entry.get()
    category = category_var.get()
    amount = amount_entry.get()
    description = description_entry.get()
    
    if not amount.replace('.', '', 1).isdigit():
        messagebox.showerror("Invalid Input", "Amount must be a number.")
        return
    
    add_expense(date, category, amount, description)
    load_expenses()
    clear_fields()



In [12]:
def load_expenses():
    for row in tree.get_children():
        tree.delete(row)
    for row in get_expenses():
        tree.insert("", tk.END, values=row)

def clear_fields():
    amount_entry.delete(0, tk.END)
    description_entry.delete(0, tk.END)


In [13]:
def generate_report():
    conn = sqlite3.connect("expenses.db")
    df = pd.read_sql_query("SELECT category, SUM(amount) as total FROM expenses GROUP BY category", conn)
    conn.close()
    
    if df.empty:
        messagebox.showinfo("No Data", "No expenses recorded yet.")
        return
    
    df.plot(kind='pie', y='total', labels=df['category'], autopct='%1.1f%%')
    plt.title("Expense Distribution")
    plt.show()


In [14]:
def generate_report():
    conn = sqlite3.connect("expenses.db")
    df = pd.read_sql_query("SELECT category, SUM(amount) as total FROM expenses GROUP BY category", conn)
    conn.close()
    
    if df.empty:
        messagebox.showinfo("No Data", "No expenses recorded yet.")
        return
    
    df.plot(kind='pie', y='total', labels=df['category'], autopct='%1.1f%%')
    plt.title("Expense Distribution")
    plt.show()


In [17]:
def export_data():
    conn = sqlite3.connect("expenses.db")
    df = pd.read_sql_query("SELECT * FROM expenses", conn)
    conn.close()
    
    file = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", ".csv"), ("Excel files", ".xlsx")])
    if file:
        if file.endswith(".csv"):
            df.to_csv(file, index=False)
        else:
            df.to_excel(file, index=False)
        messagebox.showinfo("Success", "Data exported successfully!")


In [18]:
# Buttons
ttk.Button(frame, text="Add Expense", command=add_expense_gui).grid(row=2, column=1)
ttk.Button(frame, text="Generate Report", command=generate_report).grid(row=2, column=2)
ttk.Button(frame, text="Export Data", command=export_data).grid(row=2, column=3)


In [ ]:
# Buttons
ttk.Button(frame, text="Add Expense", command=add_expense_gui).grid(row=2, column=1)
ttk.Button(frame, text="Generate Report", command=generate_report).grid(row=2, column=2)
ttk.Button(frame, text="Export Data", command=export_data).grid(row=2, column=3)

# Expense Table
tree = ttk.Treeview(frame, columns=("ID", "Date", "Category", "Amount", "Description"), show="headings")
for col in ("ID", "Date", "Category", "Amount", "Description"):
    tree.heading(col, text=col)
tree.grid(row=3, column=0, columnspan=4)

init_db()
load_expenses()
root.mainloop()